In [2]:
import numpy as np
import pandas as pd
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [3]:
# Neighborhoods in Mexico City
cdmx_cols = pd.read_csv('/Users/rogeliomj/Documents/Coursera_Capstone/Colonias/coloniascdmx.csv', sep=';')

In [4]:
cdmx_cols.head()

,COLONIA,ENTIDAD,Geo Point,Geo Shape,CVE_ALC,ALCALDIA,CVE_COL,SECC_COM,SECC_PAR
0,LOMAS DE REFORMA,9.0,"19.4016815485,-99.2354719599","{""type"": ""Polygon"", ""coordinates"": [[[-99.2359...",16,MIGUEL HIDALGO,16-043,"4971, 4983",NaN
1,DANIEL GARZA (AMPL),9.0,"19.4092184712,-99.1938392453","{""type"": ""Polygon"", ""coordinates"": [[[-99.1932...",16,MIGUEL HIDALGO,16-025,"4951, 4952, 4953","4957, 4959, 4960, 4973, 4990"
2,IGNACIO MANUEL ALTAMIRANO,9.0,"19.4631440514,-99.1968280035","{""type"": ""Polygon"", ""coordinates"": [[[-99.1966...",16,MIGUEL HIDALGO,16-034,NaN,5110
3,LEGARIA,9.0,"19.4555305454,-99.1930482732","{""type"": ""Polygon"", ""coordinates"": [[[-99.1905...",16,MIGUEL HIDALGO,16-036,5046,"5038, 5045, 5058"
4,LEGARIA (U HAB),9.0,"19.450020036,-99.2010760751","{""type"": ""Polygon"", ""coordinates"": [[[-99.2024...",16,MIGUEL HIDALGO,16-037,NaN,"5069, 5070"


In [5]:
neighborhoods = cdmx_cols[cdmx_cols['ALCALDIA'] == 'ALVARO OBREGON']
neighborhoods = neighborhoods[['ALCALDIA','COLONIA','Geo Point']]
neighborhoods.reset_index(drop=True, inplace=True)

In [6]:
geops = neighborhoods['Geo Point'].str.split(pat=',', expand=True)
neighborhoods = pd.concat([neighborhoods,geops], axis=1)
neighborhoods.drop('Geo Point', axis=1, inplace=True)
neighborhoods.rename(columns={'ALCALDIA':'District','COLONIA':'Neighborhood',0:'Latitude',1:'Longitude'}, inplace=True)
neighborhoods['Latitude'] = neighborhoods['Latitude'].astype('float')
neighborhoods['Longitude'] = neighborhoods['Longitude'].astype('float')

In [7]:
neighborhoods.head()

,District,Neighborhood,Latitude,Longitude
0,ALVARO OBREGON,2DA JALALPA TEPITO (AMPL),19.375655,-99.233207
1,ALVARO OBREGON,ESTADO DE HIDALGO_,19.364444,-99.241681
2,ALVARO OBREGON,SAN CLEMENTE,19.349309,-99.226218
3,ALVARO OBREGON,VILLA SOLIDARIDAD,19.383402,-99.219292
4,ALVARO OBREGON,TEPOPOTLA,19.362128,-99.251763


In [8]:
neighborhoods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 4 columns):
District        249 non-null object
Neighborhood    249 non-null object
Latitude        249 non-null float64
Longitude       249 non-null float64
dtypes: float64(2), object(2)
memory usage: 7.9+ KB


In [9]:
# create map of Mexico City using latitude and longitude values

latitude = 19.340995
longitude = -99.249742

map_cdmx = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='OpenStreetMap')

# add markers to map
for lat, lng, district, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], 
                                            neighborhoods['District'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx)  
    
map_cdmx

In [10]:
CLIENT_ID = 'X2BOT3ZFC3IHF0QVIQ4ZDRZC2BQBGYXOUI05KCTTS2QVUNDG' # your Foursquare ID
CLIENT_SECRET = '5IQ2I3LIBVSMU5FDBJZYZJWFVKVENIDHUGUBUD20W3H5FMXW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X2BOT3ZFC3IHF0QVIQ4ZDRZC2BQBGYXOUI05KCTTS2QVUNDG
CLIENT_SECRET:5IQ2I3LIBVSMU5FDBJZYZJWFVKVENIDHUGUBUD20W3H5FMXW


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


In [13]:
ao_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

2DA  JALALPA TEPITO (AMPL)
ESTADO DE HIDALGO_
SAN CLEMENTE
VILLA SOLIDARIDAD
TEPOPOTLA
PASEO DE LAS LOMAS-SAN GABRIEL
PRESIDENTES 2DA (AMPL)
LOS GAMITOS
LOMAS DE SANTO DOMINGO
LOMAS DE CENTENARIO (U HAB)
CANUTILLO(AGUASCALIENTES)
AGUILAS PILARES
JALALPA TEPITO
EL BATAN
BELEN DE LAS FLORES
LOMAS DE AXOMIATLA
LA PERA XOCHINAHUAC (U HAB)
MOLINO DE ROSAS
LOMAS DE LOS CEDROS
19 DE MAYO
SAN PEDRO DE LOS PINOS
SANTA LUCIA (PBLO)
PARAJE EL CABALLITO
ACUEDUCTO
SANTA MARIA NONOALCO
PRADOS LA PROVIDENCIA
SEARS ROEBUCK (U HAB)
LOMAS DE GUADALUPE
LOMAS DE CHAMONTOYA
MINAS DE CRISTO
LOMAS DE CAPULIN
MARIA G DE GARCIA RUIZ
LOMAS DE LOS ANGELES TETELPAN
MOLINO DE ROSAS (AMPL)
BARRIO NORTE
CANUTILLO 3RA SECCION
TETELPAN (PBLO)
TEPEACA
JALALPA EL GRANDE
LA CONCHITA
GALEANA
LA LOMA
LIBERACION PROLETARIA
CALZADA JALALPA
EL ARBOL
CAADA DEL OLIVAR (U HAB)
OLIVAR DE LOS PADRES
MILPA DEL CEDRO
GOLONDRINAS
EMANCIPACION DEL PUEBLO
PALMAS
SAN ANGEL INN
PUENTE COLORADO
TARANGO (U HAB)
1RA VICTORIA SECCION BOSQUES

In [14]:
print(ao_venues.shape)
ao_venues.head(10)

(4179, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,2DA JALALPA TEPITO (AMPL),19.375655,-99.233207,Campo De Futbol Jalalpa,19.374624,-99.237177,Soccer Field
1,2DA JALALPA TEPITO (AMPL),19.375655,-99.233207,La Michoacana,19.371645,-99.232556,Ice Cream Shop
2,2DA JALALPA TEPITO (AMPL),19.375655,-99.233207,El Duende,19.371489,-99.231661,Scenic Lookout
3,2DA JALALPA TEPITO (AMPL),19.375655,-99.233207,Carnitas Nacho,19.371821,-99.230726,Taco Place
4,ESTADO DE HIDALGO_,19.364444,-99.241681,Starbucks,19.365002,-99.243963,Coffee Shop
5,ESTADO DE HIDALGO_,19.364444,-99.241681,Carnitas Tamaulipas 1190,19.365284,-99.243888,Taco Place
6,ESTADO DE HIDALGO_,19.364444,-99.241681,"La Salle, Unidad Deportiva Santa Lucia",19.366259,-99.240364,Soccer Field
7,ESTADO DE HIDALGO_,19.364444,-99.241681,El Lago de los Patos,19.364521,-99.245345,Taco Place
8,ESTADO DE HIDALGO_,19.364444,-99.241681,La Barda,19.364603,-99.245235,Burger Joint
9,SAN CLEMENTE,19.349309,-99.226218,Santa Clara,19.349067,-99.226417,Ice Cream Shop


In [15]:
print('There are {} uniques categories.'.format(len(ao_venues['Venue Category'].unique())))

There are 245 uniques categories.


# Analyze each neighboorhood (Only Foursquare Data)

In [16]:
# one hot encoding
ao_onehot = pd.get_dummies(ao_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ao_onehot['Neighborhood'] = ao_venues['Neighborhood'] 

In [17]:
ao_onehot.head()

,ATM,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Wine Bar,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
ao_grouped = ao_onehot.groupby('Neighborhood').mean().reset_index()
ao_grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Wine Bar,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,19 DE MAYO,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,1RA VICTORIA,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,1RA VICTORIA SECCION BOSQUES,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,26 DE JULIO,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,2DA JALALPA TEPITO (AMPL),0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
5,2DA EL PIRUL (AMPL),0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
6,ABRAHAM GONZALEZ,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
7,ACUEDUCTO,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.055556,0.0,0.000000,0.0,0.000000
8,ACUILOTLA,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
9,AGUILAS 3ER PARQUE,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [19]:
ao_grouped.shape

(246, 245)

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in ao_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ao_grouped[ao_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----19 DE MAYO----
               venue  freq
0  Convenience Store  0.12
1         Taco Place  0.12
2   Department Store  0.12
3        Coffee Shop  0.12
4         Donut Shop  0.12


----1RA VICTORIA----
                venue  freq
0          Taco Place  0.18
1  Mexican Restaurant  0.14
2        Burger Joint  0.07
3                Park  0.07
4         Flea Market  0.04


----1RA VICTORIA SECCION BOSQUES----
        venue  freq
0  Taco Place  0.18
1        Park  0.12
2  Restaurant  0.06
3         Bar  0.06
4    Gym Pool  0.06


----26 DE JULIO----
                venue  freq
0   Convenience Store   0.2
1         Coffee Shop   0.2
2  Mexican Restaurant   0.1
3          Donut Shop   0.1
4            Pharmacy   0.1


----2DA  JALALPA TEPITO (AMPL)----
            venue  freq
0  Ice Cream Shop  0.25
1    Soccer Field  0.25
2  Scenic Lookout  0.25
3      Taco Place  0.25
4       Piano Bar  0.00


----2DA EL PIRUL (AMPL)----
                 venue  freq
0   Mexican Restaurant  0.22
1         

                venue  freq
0  Mexican Restaurant  0.33
1          Taco Place  0.17
2       Burrito Place  0.17
3               Trail  0.17
4        Cocktail Bar  0.17


----CHIMALISTAC----
                venue  freq
0  Mexican Restaurant  0.09
1                 Bar  0.06
2          Restaurant  0.04
3        Dessert Shop  0.04
4   French Restaurant  0.04


----COLINAS DE TARANGO----
                venue  freq
0  Mexican Restaurant  0.18
1         Coffee Shop  0.09
2             Dog Run  0.09
3                 Gym  0.09
4            Gym Pool  0.09


----COLINAS DEL SUR----
                venue  freq
0  Mexican Restaurant  0.20
1         Pizza Place  0.15
2        Burger Joint  0.10
3                 Gym  0.10
4   Convenience Store  0.10


----CONCIENCIA PROLETARIA----
            venue  freq
0        Mountain   0.4
1         Dog Run   0.2
2  Breakfast Spot   0.2
3             Bar   0.2
4             ATM   0.0


----COOPERATIVA CEHUAYO----
              venue  freq
0        Taco Place

4                 ATM  0.00


----JALALPA (AMPL)----
                  venue  freq
0                   Gym  0.29
1        Ice Cream Shop  0.14
2  Fast Food Restaurant  0.14
3        Farmers Market  0.14
4    Mexican Restaurant  0.14


----JALALPA EL GRANDE----
                 venue  freq
0           Taco Place  0.33
1    Convenience Store  0.17
2  Japanese Restaurant  0.17
3                 Park  0.17
4         Soccer Field  0.17


----JALALPA TEPITO----
            venue  freq
0     Supermarket   0.5
1  Farmers Market   0.5
2             ATM   0.0
3       Piano Bar   0.0
4       Nightclub   0.0


----JARDINES DEL PEDREGAL----
            venue  freq
0  Ice Cream Shop  0.16
1    Dessert Shop  0.11
2     Coffee Shop  0.11
3          Bakery  0.05
4      Restaurant  0.05


----JOSE MARIA PINO SUAREZ----
         venue  freq
0   Taco Place  0.27
1   Restaurant  0.09
2  Bus Station  0.09
3    Rest Area  0.05
4    BBQ Joint  0.05


----JURISTAS----
               venue  freq
0         Taco 

               venue  freq
0  Convenience Store  0.14
1       Burger Joint  0.14
2               Farm  0.14
3         Taco Place  0.14
4               Park  0.14


----LOMAS DE NUEVO MEXICO----
               venue  freq
0     Ice Cream Shop  0.11
1  Convenience Store  0.11
2         Taco Place  0.06
3         Laundromat  0.06
4         Food Truck  0.06


----LOMAS DE PLATEROS (U HAB) I----
                venue  freq
0          Taco Place  0.21
1  Mexican Restaurant  0.14
2      Ice Cream Shop  0.07
3                 Gym  0.07
4  Miscellaneous Shop  0.07


----LOMAS DE PLATEROS (U HAB) II----
                venue  freq
0          Taco Place  0.15
1  Mexican Restaurant  0.12
2      Ice Cream Shop  0.08
3         Coffee Shop  0.08
4                Pool  0.04


----LOMAS DE SAN ANGEL INN----
                venue  freq
0          Taco Place  0.19
1          Restaurant  0.10
2  Seafood Restaurant  0.10
3                Café  0.10
4              Bakery  0.05


----LOMAS DE SANTA FE----
  

#### Let's create a function to sort venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ao_grouped['Neighborhood']

for ind in np.arange(ao_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ao_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Venues intersection with the agebs in which they are located
#### An ageb is a defined polygon which groups several indicators

In [ ]:
v_agebs = pd.read_csv('/Users/rogeliomj/Documents/Coursera_Capstone/venues_agebs.csv')
v_agebs.rename({'Neighborho':'Neighborhood','Neighbor_1':'Latitude','Neighbor_2':'Longitude'}, axis=1, inplace=True)

In [ ]:
v_agebs.head()

#### We replace A's in te CVE_AGEB column with a random 987 number for being able to convert it to an int who can be matched to the next data set.

In [ ]:
v_agebs['CVE_AGEB'] = v_agebs['CVE_AGEB'].str.replace('A','987')

In [ ]:
v_agebs['CVE_AGEB'] = v_agebs['CVE_AGEB'].astype(int)

## Socioeconomic level in Álvaro Obregón district grouped by ageb

In [ ]:
ao_se = pd.read_csv('/Users/rogeliomj/Documents/Coursera_Capstone/ao_indic_socioeco.csv')

In [ ]:
ao_se.head()

#### The variable 'socioeconomic indicator' sums all de characteristics of each ageb, since they're registered by the percentage of the population which lacks a certain service, the smaller the value of this indicator means as higher quality of life and income.

In [ ]:
plt.hist(ao_se['total_population'])

In [ ]:
plt.hist(ao_se['socioeconomic_indicator'])

In [ ]:
ao_se.to_csv('/Users/rogeliomj/Documents/Coursera_Capstone/ao_indic_socioeco.csv', index=False)

## We merge venues and its CVE_AGEB with the socioeconomic indicator database.

In [ ]:
venues_agebs = pd.merge(ao_se,v_agebs, left_on='ageb_key',right_on='CVE_AGEB')

In [ ]:
venues_agebs.columns

In [ ]:
venues_agebs = venues_agebs.loc[:,['total_population','socioeconomic_indicator','Neighborhood','Latitude'
                                   ,'Longitude','Venue','Venue Lati','Venue Long','Venue Cate']]

In [ ]:
venues_agebs.head()

### Bining data to get population and socioeconomic indicator categories

In [ ]:
bins = [0, 5000, 10000, 15000]
venues_agebs['total_population'] = pd.cut(venues_agebs['total_population'], 
                                          bins, 
                                         labels=['low_pop_density','medium_pop_density','high_pop_density'])

In [ ]:
bins = [0, 150, 250, 400]
venues_agebs['socioeconomic_indicator'] = pd.cut(venues_agebs['socioeconomic_indicator'], 
                                          bins, 
                                          labels=['high_socio_econ_lvl','medium_socio_econ_lvl',
                                                  'low_socioecon_level'])

In [ ]:
venues_agebs.head()

### Getting dummies from venue catagories, socioeconomic indicator and total population

In [ ]:
# one hot encoding
ao_onehot = pd.get_dummies(venues_agebs[['Venue Cate','total_population','socioeconomic_indicator']],
                           prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ao_onehot['Neighborhood'] = venues_agebs['Neighborhood']

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
ao_grouped = ao_onehot.groupby('Neighborhood').mean().reset_index()
ao_grouped

In [ ]:
ao_grouped_clustering = ao_grouped.drop('Neighborhood', 1)

In [ ]:
ao_grouped_clustering.head()

## Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ao_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
ao_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
ao_grouped.head()

In [ ]:
ao_grouped = pd.merge(ao_grouped,neighborhoods, on=['Neighborhood'], how='inner')

In [ ]:
ao_grouped.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12, tiles="OpenStreetMap")

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ao_grouped['Latitude'], ao_grouped['Longitude'], ao_grouped['Neighborhood'],
                                  ao_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster analysis

### Cluster 0

In [ ]:
ao_grouped['Cluster Labels'].unique()

In [ ]:
cl0 = ao_grouped[ao_grouped['Cluster Labels'] == 1]

#### Dataframe only with venues

In [ ]:
cl0_venues = cl0.drop(['low_pop_density',
                           'medium_pop_density','high_pop_density',
                           'high_socio_econ_lvl','medium_socio_econ_lvl',
                           'low_socioecon_level','District','Latitude',
                           'Longitude','Cluster Labels'], axis=1)


In [ ]:
num_top_venues = 5

for hood in cl0_venues['Neighborhood']:
    print("----"+hood+"----")
    temp = cl0_venues[cl0_venues['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cl0_venues['Neighborhood']

for ind in np.arange(cl0_venues.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cl0_venues.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Dataframe only with population indicators

In [ ]:
cl0_pop_indicators = cl0.loc[:,['Neighborhood','low_pop_density',
                           'medium_pop_density','high_pop_density']]

In [ ]:
num_top_pops = 3

for hood in cl0_pop_indicators['Neighborhood']:
    print("----"+hood+"----")
    temp = cl0_pop_indicators[cl0_pop_indicators['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_pops))
    print('\n')

In [ ]:
def return_most_common_pops(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_pops = 2

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_pops):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_pops_sorted = pd.DataFrame(columns=columns)
neighborhoods_pops_sorted['Neighborhood'] = cl0_pop_indicators['Neighborhood']

for ind in np.arange(cl0_venues.shape[0]):
    neighborhoods_pops_sorted.iloc[ind, 1:] = return_most_common_pops(cl0_pop_indicators.iloc[ind, :],
                                                                          num_top_pops)

neighborhoods_pops_sorted.head()

#### Dataframe only with socioeconomic indicators

In [ ]:
cl0_indicators = cl0.loc[:,['Neighborhood','high_socio_econ_lvl','medium_socio_econ_lvl','low_socioecon_level']]

In [ ]:
num_top_inds = 5

for hood in cl0_indicators['Neighborhood']:
    print("----"+hood+"----")
    temp = cl0_indicators[cl0_indicators['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_pops))
    print('\n')

In [ ]:
def return_most_common_inds(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_inds = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_inds):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_inds_sorted = pd.DataFrame(columns=columns)
neighborhoods_inds_sorted['Neighborhood'] = cl0_indicators['Neighborhood']

for ind in np.arange(cl0_venues.shape[0]):
    neighborhoods_inds_sorted.iloc[ind, 1:] = return_most_common_inds(cl0_indicators.iloc[ind, :],
                                                                          num_top_inds)

neighborhoods_inds_sorted.head()